In [1]:
import numpy as np
import cv2
import csv
from pynq import allocate
from pynq import Overlay

ol = Overlay('bitstream/final_v1_45_encrypt.bit') 
ol?

In [3]:
# #testing few stuff
# test_val = np.int64(-99999)
# test_all = allocate(shape=(1,), dtype=np.uint32)
# test_all[0] = test_val
# print(type(test_all[0]))
# print(test_all)

In [3]:
#loading the pattern from .csv and choosing the 1

patterns = []  # each item is a list of 0/1 for that digit (digit 1-9)

with open("NEST_output/normalized_matrix.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)  # Load all rows into memory

    if rows:
        pattern_length = len(rows[0])  # Get the number of bits per pattern
#         patterns.append([0] * pattern_length)  # Add the 0th pattern (all zeros)

        for row in rows:
            pattern = [int(float(bit.strip())) for bit in row]
            patterns.append(pattern)
            
print(len(patterns[0]))
print(patterns)

choosen_ones = [0,59,39,45,37,50,44,52,41,54]
print(choosen_ones)

61
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

In [4]:
def pad_and_chunk_pattern(pattern):
    padded = pattern + [0] * (64 - len(pattern))  # add 3 zeros
    bitstring = "".join(str(bit) for bit in padded[::-1])  # reverse the bit list
    chunk_high = int(bitstring[32:64], 2)  # bits 32 to 63
    chunk_low =  int(bitstring[0:32], 2)  # bits 0 to 31
    return chunk_high, chunk_low

def pack_choosen_values(choosen_list):
    assert len(choosen_list) == 10
    chunks = []
    for i in range(0, 9, 3):  # process 3 values per chunk
        val0 = choosen_list[i]     & 0x1FF  # 9 bits
        val1 = choosen_list[i + 1] & 0x1FF
        val2 = choosen_list[i + 2] & 0x1FF

        chunk = (val2 << 18) | (val1 << 9) | val0  # pack into 0–26 bits
        chunks.append(chunk)
    chunks.append(choosen_list[9] & 0x1FF)
    return chunks

packed_chunks = []  # list of (high, low) tuples

for pattern in patterns:
    high, low = pad_and_chunk_pattern(pattern)
    packed_chunks.append((high, low))
    

print(packed_chunks)

choosen_chunks = pack_choosen_values(choosen_ones)
print(choosen_chunks)

[(0, 0), (0, 134217728), (0, 134217856), (2147483648, 134225920), (67108864, 268501024), (8388608, 134480385), (269484032, 135270416), (33816576, 135282817), (545390592, 136348168), (69238784, 138479649)]
[10253824, 13126189, 10774572, 54]


In [6]:
# #image analysis

img_path = "input_images/7.png"
# Load image
img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)  # Preserve alpha channel

if img is None:
    print(f"Skipping {img_file}: Unable to read")
    exit()

if img.shape[2] != 4:
    print(f"Skipping {img_path}: Not a 4-channel image")
    exit()

# Get image details
height, width = img.shape[:2]
channels = img.shape[2]
dtype = img.dtype  # Data type of image pixels

# Print image details
print(f"Image Shape: {img.shape}")
print(f"Image Width: {width}, Height: {height}")
print(f"Number of Channels: {channels}")
print(f"Data Type: {dtype}")  

Image Shape: (1080, 1920, 4)
Image Width: 1920, Height: 1080
Number of Channels: 4
Data Type: uint8


In [7]:
# # # sound stuff
sound_data = np.load("sound/data1.npy")  # Shape: (N, 2)

flattened_data = sound_data.flatten()  # Convert to 1D (interleaved left-right channels)
num_pixels = height * width
num_sound_values = num_pixels // 3  # One sound value per 3 pixels
num_entries = num_pixels + num_sound_values

if len(flattened_data) < num_sound_values:
    # Pad with 0s if sound data is smaller
    flattened_data = np.concatenate([flattened_data, np.zeros(num_sound_values - len(flattened_data), dtype=np.int64)])

if len(flattened_data) > num_sound_values:
    flattened_data = flattened_data[:num_sound_values]
    print("Can't fit the whole sound in image it is sliced")
    
print(flattened_data.dtype)
print(num_sound_values)

int64
691200


[0 0 0 0 0 0 0 0 0 0]
(691200,)


In [8]:
img_flat = img.reshape(-1, 4).astype(np.uint32)
bgra_values = (img_flat[:, 0] << 24) | (img_flat[:, 1] << 16) | (img_flat[:, 2] << 8) | img_flat[:, 3]

input_buffer = allocate(shape=(num_entries + 24,), dtype=np.uint32)
output_buffer = allocate(shape=(height * width + 2,), dtype=np.uint32)

interleaved = np.zeros(num_entries, dtype=np.uint32)

sound_positions = np.arange(3, num_entries, 4)
img_positions = np.delete(np.arange(num_entries), sound_positions)

interleaved[img_positions[:len(bgra_values)]] = bgra_values

interleaved[sound_positions[:len(flattened_data)]] = flattened_data[:len(sound_positions)]

input_buffer[24:] = interleaved

print("Data embedded successfully!")

Data embedded successfully!


In [9]:
print(img[0,0])
print(input_buffer)
print(output_buffer)
print(input_buffer.dtype)

[  0   0   1 255]
[        0         0         0 ... 370489343      4863         0]
[0 0 0 ... 0 0 0]
uint32


In [10]:
#writing initial 24 values
for i, chunk in enumerate(packed_chunks):
    input_buffer[2 * i] = chunk[0]
    input_buffer[2 * i + 1] = chunk[1]

input_buffer[20] = choosen_chunks[0]
input_buffer[21] = choosen_chunks[1]
input_buffer[22] = choosen_chunks[2]
input_buffer[23] = choosen_chunks[3]

print(input_buffer)
print(num_entries)
print(len(input_buffer))

[        0         0         0 ... 370489343      4863         0]
2764800
2764824


In [11]:
dma = ol.axi_dma_0

In [12]:
#FLUSH & INVALIDATE BUFFERS
input_buffer.flush()         # Ensure data is written to memory before DMA reads it
output_buffer.invalidate()   # Ensure CPU reads fresh data after DMA writes it


In [13]:
# dma.sendchannel.stop()
# dma.recvchannel.stop()

# dma.register_map.MM2S_DMACR.Reset = 1  # Reset MM2S (Memory to Stream)
# dma.register_map.S2MM_DMACR.Reset = 1  # Reset S2MM (Stream to Memory)


# while dma.register_map.MM2S_DMACR.Reset:
#     pass  
# while dma.register_map.S2MM_DMACR.Reset:
#     pass

# dma.sendchannel.start()
# dma.recvchannel.start()

# print("DMA S2MM Status:", hex(dma.register_map.S2MM_DMASR))
# print("DMA MM2S Status:", hex(dma.register_map.MM2S_DMASR))
# print("Send Channel Running:", dma.sendchannel.running)
# print("Receive Channel Running:", dma.recvchannel.running)

In [13]:
# dma.sendchannel.transfer(input_buffer)
dma.sendchannel.transfer(input_buffer)
dma.recvchannel.transfer(output_buffer)
print("Send Channel Idle:", dma.sendchannel.idle)
print("Receive Channel Idle:", dma.recvchannel.idle)
print("MM2S Transfer Length:", hex(dma.register_map.MM2S_LENGTH))
print("DMA S2MM Status:", hex(dma.register_map.S2MM_DMASR))
print("DMA MM2S Status:", hex(dma.register_map.MM2S_DMASR))

dma.sendchannel.wait()
dma.recvchannel.wait()

Send Channel Idle: False
Receive Channel Idle: False
MM2S Transfer Length: 0xa8c060
DMA S2MM Status: 0x1002
DMA MM2S Status: 0x1002


In [14]:
print(input_buffer)
print(output_buffer)
print(output_buffer.dtype)

[        0         0         0 ... 370489343      4863         0]
[       116 1395724288        252 ...      12540  336868604       4348]
uint32


In [15]:
print(img[0,:18,:])
print(input_buffer[:24])
print(output_buffer[:24])

[[  0   0   1 255]
 [  5   4   6 255]
 [ 11   6   5 255]
 [ 10   0   0 255]
 [ 13   2   0 255]
 [ 14   3   0 255]
 [ 10   4   0 255]
 [ 11   5   0 255]
 [ 12   4   0 255]
 [ 12   4   0 255]
 [ 12   3   0 255]
 [ 12   3   0 255]
 [ 10   4   0 255]
 [ 12   3   0 255]
 [ 12   4   0 255]
 [ 14   3   0 255]
 [ 13   4   0 255]
 [ 13   4   1 255]]
[         0          0          0  134217728          0  134217856
 2147483648  134225920   67108864  268501024    8388608  134480385
  269484032  135270416   33816576  135282817  545390592  136348168
   69238784  138479649   10253824   13126189   10774572         54]
[       116 1395724288        252   67372284  201852156  134217980
  201326844  268697852  134480124  201588988  201588988  201588988
  201588988  201588988  134480124  201588988  201588988  201326844
  201588988  201588988  201588988  201588988  201588988  201588988]


In [16]:
print("Send Channel Idle:", dma.sendchannel.idle)
print("Receive Channel Idle:", dma.recvchannel.idle)

Send Channel Idle: True
Receive Channel Idle: True


In [16]:
#saving the key and changing the ouptut buffer
key = []
key.append(output_buffer[0])
key.append(output_buffer[1])
print(key)

key_array = np.array(key)
np.save("key.npy", key_array)

new_output_buffer = output_buffer[2:]
print(new_output_buffer)
print(len(new_output_buffer))
print(new_output_buffer.dtype)

[116, 1395724288]
[      252  67372284 201852156 ...     12540 336868604      4348]
2073600
uint32


In [18]:
# Reconstruct image from 32-bit output buffer

image_flat = new_output_buffer[:height * width]

B = ((image_flat >> 24) & 0xFF).astype(np.uint8)
G = ((image_flat >> 16) & 0xFF).astype(np.uint8)
R = ((image_flat >> 8) & 0xFF).astype(np.uint8)
A = (image_flat & 0xFF).astype(np.uint8)

reconstructed_img = np.stack([B, G, R, A], axis=1).reshape((height, width, 4))

cv2.imwrite("encrypted_images/encrypt_7.png", reconstructed_img)

print("Encrypted image saved as ---------.png")

Encrypted image saved as ---------.png
